In [1]:
!pip install -q gradio

In [2]:
!mkdir ./tmp
!gsutil -m cp -r \
  "gs://gcp-ml-172005-complete-mlops/tfx_pipeline_output/segformer-training-pipeline/874401645461/segformer-training-pipeline-20220910042819/Trainer_-9039145015330209792/model/Format-Serving" \
  ./tmp

mkdir: cannot create directory ‘./tmp’: File exists
Copying gs://gcp-ml-172005-complete-mlops/tfx_pipeline_output/segformer-training-pipeline/874401645461/segformer-training-pipeline-20220910042819/Trainer_-9039145015330209792/model/Format-Serving/variables/variables.index...
Copying gs://gcp-ml-172005-complete-mlops/tfx_pipeline_output/segformer-training-pipeline/874401645461/segformer-training-pipeline-20220910042819/Trainer_-9039145015330209792/model/Format-Serving/variables/variables.data-00000-of-00001...
Copying gs://gcp-ml-172005-complete-mlops/tfx_pipeline_output/segformer-training-pipeline/874401645461/segformer-training-pipeline-20220910042819/Trainer_-9039145015330209792/model/Format-Serving/keras_metadata.pb...
Copying gs://gcp-ml-172005-complete-mlops/tfx_pipeline_output/segformer-training-pipeline/874401645461/segformer-training-pipeline-20220910042819/Trainer_-9039145015330209792/model/Format-Serving/saved_model.pb...


In [3]:
model_path = "model"

!mkdir {model_path}
!cp -r tmp/Format-Serving/* {model_path}/

mkdir: cannot create directory ‘model’: File exists


In [4]:
!saved_model_cli show --dir {model_path} --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['string_input'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_string_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['seg_mask'] tensor_info:
      dtype: DT_INT64
      shape: (-1, 128, 128)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [12]:
from PIL import Image 
import numpy as np 
import tensorflow as tf 


RESOLTUION = 128
MODEL = tf.keras.models.load_model(model_path)


def preprocess_input(image: Image) -> tf.Tensor:
    image = np.array(image)
    image = tf.convert_to_tensor(image)

    image = tf.image.resize(image, (RESOLTUION, RESOLTUION))
    image = image / 255

    return tf.expand_dims(image, 0)


# The below utilities (sidewalk_palette(), get_seg_overlay()) are from:
# https://github.com/deep-diver/semantic-segmentation-ml-pipeline/blob/main/notebooks/inference_from_SavedModel.ipynb

def sidewalk_palette():
    """Sidewalk palette that maps each class to RGB values."""
    return [
        [0, 0, 0],
        [216, 82, 24],
        [255, 255, 0],
        [125, 46, 141],
        [118, 171, 47],
        [161, 19, 46],
        [255, 0, 0],
        [0, 128, 128],
        [190, 190, 0],
        [0, 255, 0],
        [0, 0, 255],
        [170, 0, 255],
        [84, 84, 0],
        [84, 170, 0],
        [84, 255, 0],
        [170, 84, 0],
        [170, 170, 0],
        [170, 255, 0],
        [255, 84, 0],
        [255, 170, 0],
        [255, 255, 0],
        [33, 138, 200],
        [0, 170, 127],
        [0, 255, 127],
        [84, 0, 127],
        [84, 84, 127],
        [84, 170, 127],
        [84, 255, 127],
        [170, 0, 127],
        [170, 84, 127],
        [170, 170, 127],
        [170, 255, 127],
        [255, 0, 127],
        [255, 84, 127],
        [255, 170, 127],
    ]


def get_seg_overlay(image, seg):
    color_seg = np.zeros(
        (seg.shape[0], seg.shape[1], 3), dtype=np.uint8
    )  # height, width, 3
    palette = np.array(sidewalk_palette())
    
    for label, color in enumerate(palette):
        color_seg[seg == label, :] = color

    # Show image + mask
    img = np.array(image) * 0.5 + color_seg * 0.5

    img *= 255
    img = np.clip(img, 0, 255)
    img = img.astype(np.uint8)
    return img


def run_model(image: Image) -> tf.Tensor:
    preprocessed_image = preprocess_input(image)
    prediction = MODEL.predict(preprocessed_image)

    seg_mask = tf.math.argmax(prediction, -1)
    seg_mask = tf.squeeze(seg_mask)
    return seg_mask


def get_predictions(image: Image):
    predicted_segmentation_mask = run_model(image)
    preprocessed_image = preprocess_input(image)
    preprocessed_image = tf.squeeze(preprocessed_image, 0)
    
    pred_img = get_seg_overlay(preprocessed_image.numpy(), predicted_segmentation_mask.numpy())
    return Image.fromarray(pred_img)

In [ ]:
import gradio as gr

title = "Simple demo for a semantic segmentation model trained on the Sidewalks dataset."

description = """

Note that the outputs obtained in this demo won't be state-of-the-art. The underlying project has a different objective focusing more on the ops side of
deploying a semantic segmentation model. For more details, check out the repository: https://github.com/deep-diver/semantic-segmentation-ml-pipeline/.

"""

demo = gr.Interface(
    get_predictions,
    gr.inputs.Image(type="pil"),
    "pil",
    allow_flagging="never",
    title=title,
    description=description,
)

demo.launch(debug=True)

/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://22644.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces
